In [10]:
import pandas as pd
import numpy as np

set171 = ['170506','170513','170520', '170527','170603']
list171=[pd.DataFrame()]*5;
for i in range(5):
    str17 = "http://web.mta.info/developers/data/nyct/turnstile/turnstile_" + set171[i] + ".txt"
    list171[i] = pd.read_csv(str17)
y171 = pd.concat(list171)
y171.columns = [column.strip() for column in y171.columns]

In [14]:
# We can drop our index if we chose to.

y171=y171.reset_index()
y171=y171.drop('index',1)
y171.tail()

,C/A,UNIT,SCP,STATION,LINENAME,DIVISION,DATE,TIME,DESC,ENTRIES,EXITS
981042,TRAM2,R469,00-05-01,RIT-ROOSEVELT,R,RIT,06/02/2017,05:00:00,REGULAR,5554,299
981043,TRAM2,R469,00-05-01,RIT-ROOSEVELT,R,RIT,06/02/2017,09:00:00,REGULAR,5554,299
981044,TRAM2,R469,00-05-01,RIT-ROOSEVELT,R,RIT,06/02/2017,13:00:00,REGULAR,5554,299
981045,TRAM2,R469,00-05-01,RIT-ROOSEVELT,R,RIT,06/02/2017,17:00:00,REGULAR,5554,299
981046,TRAM2,R469,00-05-01,RIT-ROOSEVELT,R,RIT,06/02/2017,21:00:00,REGULAR,5554,299


In [13]:
# We can drop our index if we chose to.

#turnstiles_df=turnstiles_df.reset_index()
#turnstiles_df=turnstiles_df.drop('index',1)
#turnstiles_df.tail()

,C/A,UNIT,SCP,STATION,LINENAME,DIVISION,DATE,TIME,DESC,ENTRIES,EXITS
195953,TRAM2,R469,00-05-01,RIT-ROOSEVELT,R,RIT,06/02/2017,05:00:00,REGULAR,5554,299
195954,TRAM2,R469,00-05-01,RIT-ROOSEVELT,R,RIT,06/02/2017,09:00:00,REGULAR,5554,299
195955,TRAM2,R469,00-05-01,RIT-ROOSEVELT,R,RIT,06/02/2017,13:00:00,REGULAR,5554,299
195956,TRAM2,R469,00-05-01,RIT-ROOSEVELT,R,RIT,06/02/2017,17:00:00,REGULAR,5554,299
195957,TRAM2,R469,00-05-01,RIT-ROOSEVELT,R,RIT,06/02/2017,21:00:00,REGULAR,5554,299


In [15]:
y171['DATETIME'] = [pd.to_datetime(i+j, format = '%m/%d/%Y%H:%M:%S') for i, j in zip(y171.DATE, y171.TIME)]

,C/A,UNIT,SCP,STATION,LINENAME,DIVISION,DATE,TIME,DESC,ENTRIES,EXITS,DATETIME
0,A002,R051,02-00-00,59 ST,NQR456W,BMT,04/29/2017,00:00:00,REGULAR,6157740,2085315,2017-04-29 00:00:00
1,A002,R051,02-00-00,59 ST,NQR456W,BMT,04/29/2017,04:00:00,REGULAR,6157777,2085319,2017-04-29 04:00:00
2,A002,R051,02-00-00,59 ST,NQR456W,BMT,04/29/2017,08:00:00,REGULAR,6157810,2085353,2017-04-29 08:00:00
3,A002,R051,02-00-00,59 ST,NQR456W,BMT,04/29/2017,12:00:00,REGULAR,6157963,2085453,2017-04-29 12:00:00
4,A002,R051,02-00-00,59 ST,NQR456W,BMT,04/29/2017,16:00:00,REGULAR,6158212,2085529,2017-04-29 16:00:00


In [19]:
(y171
 .groupby(["C/A", "UNIT", "SCP", "STATION", "DATE", "DATETIME"])
 .ENTRIES.count()
 .reset_index()
 .sort_values("ENTRIES", ascending=False)).head(5)

,C/A,UNIT,SCP,STATION,DATE,DATETIME,ENTRIES
871022,R504,R276,00-00-01,VERNON-JACKSON,05/26/2017,2017-05-26 20:00:00,2
135038,G001,R151,00-00-00,CONEY IS-STILLW,05/09/2017,2017-05-09 05:00:00,2
0,A002,R051,02-00-00,59 ST,04/29/2017,2017-04-29 00:00:00,1
654035,R145,R032,00-00-03,TIMES SQ-42 ST,05/13/2017,2017-05-13 12:00:00,1
654024,R145,R032,00-00-03,TIMES SQ-42 ST,05/11/2017,2017-05-11 16:00:00,1


In [26]:
mask = ((y171["C/A"] == "R504") & 
(y171["UNIT"] == "R276") & 
(y171["SCP"] == "00-00-01") & 
(y171["STATION"] == "VERNON-JACKSON") &
(y171["DATE"] == "05/26/2017"))
y171[mask]
y171.drop(y171.index[763079], inplace = True)
y171[mask]
mask = ((y171["C/A"] == "G001") & 
(y171["UNIT"] == "R151") & 
(y171["SCP"] == "00-00-00") & 
(y171["STATION"] == "CONEY IS-STILLW") &
(y171["DATE"] == "05/09/2017"))
y171[mask]
y171.drop(y171.index[224251], inplace = True)
y171[mask]

/Users/subramanianiyer/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:16: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  app.launch_new_instance()


,C/A,UNIT,SCP,STATION,LINENAME,DIVISION,DATE,TIME,DESC,ENTRIES,EXITS,DATETIME
224250,G001,R151,00-00-00,CONEY IS-STILLW,DFNQ,BMT,05/09/2017,01:00:00,REGULAR,199982,337867,2017-05-09 01:00:00
224252,G001,R151,00-00-00,CONEY IS-STILLW,DFNQ,BMT,05/09/2017,05:00:00,RECOVR AUD,200021,337947,2017-05-09 05:00:00
224253,G001,R151,00-00-00,CONEY IS-STILLW,DFNQ,BMT,05/09/2017,09:00:00,REGULAR,200363,338280,2017-05-09 09:00:00
224254,G001,R151,00-00-00,CONEY IS-STILLW,DFNQ,BMT,05/09/2017,13:00:00,REGULAR,200546,338821,2017-05-09 13:00:00
224255,G001,R151,00-00-00,CONEY IS-STILLW,DFNQ,BMT,05/09/2017,17:00:00,REGULAR,200860,339305,2017-05-09 17:00:00
224256,G001,R151,00-00-00,CONEY IS-STILLW,DFNQ,BMT,05/09/2017,21:00:00,REGULAR,201098,339691,2017-05-09 21:00:00


,C/A,UNIT,SCP,STATION,DATE,DATETIME,EXITS
0,A002,R051,02-00-00,59 ST,04/29/2017,2017-04-29 00:00:00,1
654035,R145,R032,00-00-03,TIMES SQ-42 ST,05/13/2017,2017-05-13 12:00:00,1
654023,R145,R032,00-00-03,TIMES SQ-42 ST,05/11/2017,2017-05-11 12:00:00,1
654024,R145,R032,00-00-03,TIMES SQ-42 ST,05/11/2017,2017-05-11 16:00:00,1
654025,R145,R032,00-00-03,TIMES SQ-42 ST,05/11/2017,2017-05-11 20:00:00,1


In [37]:
y171 = y171.reset_index()

In [38]:
del y171['DESC']
y171['ENTRIESDOWN'] = [int(j) for j in y171.groupby(['C/A','UNIT','SCP','STATION','LINENAME','DIVISION']).ENTRIES.shift()]
y171['EXITSDOWN'] = [int(j) for j in y171.groupby(['C/A','UNIT','SCP','STATION','LINENAME','DIVISION']).EXITS.shift()]
y171.dropna(inplace = True)
y171.head()

,index,C/A,UNIT,SCP,STATION,LINENAME,DIVISION,DATE,TIME,ENTRIES,EXITS,DATETIME,ENTRIESDOWN,EXITSDOWN
1,1,A002,R051,02-00-00,59 ST,NQR456W,BMT,04/29/2017,04:00:00,6157777,2085319,2017-04-29 04:00:00,6157740,2085315
2,2,A002,R051,02-00-00,59 ST,NQR456W,BMT,04/29/2017,08:00:00,6157810,2085353,2017-04-29 08:00:00,6157777,2085319
3,3,A002,R051,02-00-00,59 ST,NQR456W,BMT,04/29/2017,12:00:00,6157963,2085453,2017-04-29 12:00:00,6157810,2085353
4,4,A002,R051,02-00-00,59 ST,NQR456W,BMT,04/29/2017,16:00:00,6158212,2085529,2017-04-29 16:00:00,6157963,2085453
5,5,A002,R051,02-00-00,59 ST,NQR456W,BMT,04/29/2017,20:00:00,6158521,2085589,2017-04-29 20:00:00,6158212,2085529


In [47]:
y171['ENTRIES'] = [i - int(j) for i,j in zip(y171['ENTRIES'], y171['ENTRIESDOWN'])]
y171['EXITS'] = [i - int(j) for i,j in zip(y171['EXITS'], y171['EXITSDOWN'])]


In [66]:
mask = ((y171["ENTRIES"] > -1) & 
(y171["EXITS"] > -1))
y171 = y171[mask]
mask = ((y171["ENTRIES"]<144000)&
       (y171["EXITS"]<144000))
y171 = y171[mask]

In [61]:
del y171['ENTRIESDOWN']
del y171['EXITSDOWN']

In [71]:
np.max(y171['EXITS'])

127234

In [73]:
y171.to_csv('Project1BestData.csv')